In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCNLinkPredictor(nn.Module):
    def __init__(self, num_in_features, num_out_features=128, num_users=1):
        super().__init__()
        self.conv1 = GCNConv(num_in_features, num_out_features)
        self.conv2 = GCNConv(num_out_features, num_out_features)
        self.num_users = num_users

    def forward(self, x, edge_index):
        user_embedding = x[:self.num_users]
        movie_features = x[self.num_users:].detach()
        
        x = torch.cat([user_embedding, movie_features], dim=0)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        
        x[self.num_users:] = movie_features
        return x

# 유저 - 영화 간 edge를 입력받아 유저-영화 간 관계 예측
class LinkPredictor(nn.Module):
    # 비선형으로 개선
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim * 2, input_dim)
        self.fc2 = nn.Linear(input_dim, 1)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, z, edge_index):
        row, col = edge_index
        z_row = z[row]
        z_col = z[col]
        
        # 임베딩 연결
        z_concat = torch.cat([z_row, z_col], dim=1)
        
        # 비선형 활성화 함수
        x = F.relu(self.fc1(z_concat))
        
        # 정규화를 위한 드롭아웃
        x = self.dropout(x)
        
        # 두 번째 선형 계층 및 시그모이드 활성화 적용
        return torch.sigmoid(self.fc2(x))

In [2]:
num_in_features = 404
num_out_features = 404
num_users = 1

# load saved models
gcn_model = GCNLinkPredictor(num_in_features, num_out_features, num_users)
link_predictor = LinkPredictor(num_out_features)
gcn_model.load_state_dict(torch.load('gcn_model.pth'))
link_predictor.load_state_dict(torch.load('link_predictor.pth'))

# 모델을 evaluation 모드로 변경
gcn_model.eval()
link_predictor.eval()

LinkPredictor(
  (fc1): Linear(in_features=808, out_features=404, bias=True)
  (fc2): Linear(in_features=404, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [3]:
import pandas as pd

movie_metadata = pd.read_excel("cinemate_data_0511.xlsx")
final_metadata = pd.read_excel("final_metadata_with_overview_embeddings_ver5_0511.xlsx")

In [4]:
import numpy as np

# 유저 임베딩 생성 함수
def create_new_user_embedding(movie_features, interacted_movie_indices):
    new_user_embedding = movie_features[interacted_movie_indices].mean(dim=0)
    # print("new user embedding shape:", new_user_embedding.shape)
    # print("new user embedding:", new_user_embedding)
    return new_user_embedding

# 새로운 유저에 대한 영화 추천
def recommend_movies_for_new_user(node_embeddings, num_users = 1, num_movies = 9525, num_recommendations=5):
    movie_indices = torch.arange(num_users, num_users + num_movies) # 영화 인덱스 생성 (유저 수만큼 offset)
    
    # user-movie pairs 생성
    user_movie_pairs = torch.stack([torch.zeros(num_movies, dtype=torch.long), movie_indices], dim=0)
    print(user_movie_pairs)
    
    # user-movie pairs의 score 계산
    scores = link_predictor(node_embeddings, user_movie_pairs)
    # print(scores[scores > 0.9])
    
    # top N 추천
    _, top_indices = torch.topk(scores.squeeze(), num_recommendations)
    top_movie_indices = user_movie_pairs[1][top_indices] - num_users # 유저 수만큼 offset 재조정
    
    return top_movie_indices


# 영화 features
movie_features = torch.from_numpy(np.load("final_features_without_directors_0518.npy")).float()

# 새로운 유저가 6076, 9519 영화를 시청했다고 가정
# new_user_interacted_movies = [6076, 9519] # 점퍼, 메트로폴리스
# new_user_interacted_movies = [4887, 5839] # 7번 방의 선물, 세 얼간이
new_user_interacted_movies = [6, 10, 14] # Citizen of a Kind, It's Okay!, Deurim (코미디들)
new_user_embedding = create_new_user_embedding(movie_features, new_user_interacted_movies)

new_x = torch.cat([new_user_embedding.view(1, -1), movie_features], dim=0)
num_users = 1

# 유저 - 영화 간 상호작용 edge index로 변환
user_indices = [i for i in range(num_users)]
movie_indices = [i + num_users for i in new_user_interacted_movies]

edge_index = torch.tensor([user_indices * len(movie_indices), movie_indices], dtype=torch.long)

node_embeddings = gcn_model(new_x, edge_index)

# 새로운 유저에 대한 추천 수행
num_recommendations = 20
top_movies_for_new_user = recommend_movies_for_new_user(node_embeddings, num_recommendations=num_recommendations)
print(f"Top {num_recommendations} recommended movies for the new user: {top_movies_for_new_user}")
for idx in top_movies_for_new_user:
    print(final_metadata.iloc[int(idx)].genres)
    print(movie_metadata.iloc[int(idx)].overview)

tensor([[   0,    0,    0,  ...,    0,    0,    0],
        [   1,    2,    3,  ..., 9523, 9524, 9525]])
Top 20 recommended movies for the new user: tensor([4226, 5667, 1111,   55, 5804, 4737, 5276, 2260, 7599, 6529, 7706, 3675,
        4769, 5340, 3504, 1862, 9212, 8410, 5787, 6697])
Action,Crime
화려한 언변, 사람을 현혹하는 재능, 정관계를 넘나드는 인맥으로 수만 명 회원들에게 사기를 치며 승승장구해 온 원네트워크 진회장. 반년간 그를 추적해 온 지능범죄수사팀장 김재명은 진회장의 최측근인 박장군을 압박한다. 원네트워크 전산실 위치와 진회장의 로비 장부를 넘기라는 것. 뛰어난 프로그래밍 실력과 명석한 두뇌로 원네트워크를 키워 온 브레인 박장군은 계획에 차질이 생긴 것을 감지하자 빠르게 머리를 굴리기 시작한다. 진회장은 물론 그의 뒤에 숨은 권력까지 모조리 잡기 위해 포위망을 좁혀가는 재명,  오히려 이 기회를 틈타 돈도 챙기고 경찰의 압박에서도 벗어날 계획을 세우는 장군. 하지만 진회장은 간부 중에 배신자가 있음을 눈치채고, 새로운 플랜을 가동하는데…
Action,Crime,Drama
한 때 잘나가던 전직 형사이자 지금은 흥신소를 운영하는 강태식. 평범한 의뢰라고 생각하고 급습한 불륜 현장에 한 여자가 죽어 있다. 꼼짝없이 범인으로 몰리게 된 그 때, 걸려오는 전화 한 통 살인 누명을 벗으려면 누군가를 납치하라는 놈의 지시. 숨 돌릴 틈 없이 시작된 경찰의 추격, 자신의 일거수일투족은 물론, 과거 사연, 그리고 주변 인물까지 장악하고 있는 놈의 감시와 도청; 게다가 납치해야 하는 인물이 전국을 떠들썩하게 할 중요한 사건의 키를 쥐고 있다는 사실을 알게 된다. 이제, 자신을 조종하려는 놈과 실체를 알 수 없는 배후에 맞서 폭풍 같